In [ ]:
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim
import anndata as ad



In [ ]:

path = '../../data/TS.h5ad'
ts_ann_data = ad.read_h5ad(path)


In [ ]:

ts_df = ts_ann_data.to_df()
# Drop zero columns
ts_df = ts_df.loc[:, (ts_df != 0).any(axis=0)]
ts_df

In [ ]:
train_df, test_df = train_test_split(ts_df, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=42)

In [ ]:
print(len(train_df))
print(len(val_df))
print(len(test_df))

In [ ]:
class TSDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):

        features = torch.tensor(self.df[idx], dtype=torch.float32)

        if self.transform:
            features = self.transform(features)

        return features, features # for Autoencoder

In [ ]:
# Set hyperparameters
input_size = len(ts_df.columns)
encoding_size = 5000
learning_rate = 1e-4
epochs = 50
batch_size = 265

In [ ]:
train_dataset = TSDataset(train_df)
val_dataset = TSDataset(val_df)
test_dataset = TSDataset(test_df)

# Define DataLoader for each set
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Define Autoencoder model
class Autoencoder(nn.Module):
    def __init__(self, input_size, encoding_size):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Linear(input_size, encoding_size)
        self.decoder = nn.Linear(encoding_size, input_size)
        self.nonlin = nn.ReLU()

    def forward(self, x):
        x = self.encoder(x)
        x = self.nonlin(x)
        x = self.decoder(x)
        x = self.nonlin(x)
        return x

In [ ]:
# Initialize model, loss function, and optimizer
model = Autoencoder(input_size, encoding_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    # Train step
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{epochs}], Train Loss: {loss.item():.4f}')

    # Val step
    with torch.no_grad():
        for inputs, targets in train_loader:
            model.eval()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
        print(f'Validation Loss: {loss.item():.4f}')